# 方策勾配法の収束レートについて

参考：
* [On the Theory of Policy Gradient Methods: Optimality, Approximation, and Distribution Shift](https://arxiv.org/abs/1908.00261)

Occupancy measureを
$$
d_{s_0}^\pi(s):=(1-\gamma) \sum_{t=0}^{\infty} \gamma^t \operatorname{Pr}^\pi\left(s_t=s \mid s_0\right)
$$
とします．このとき，方策勾配は

$$
\nabla_\theta V^{\pi_\theta}\left(s_0\right)=\frac{1}{1-\gamma} \mathbb{E}_{s \sim d_{\sigma_0}^{\pi_\theta}} \mathbb{E}_{a \sim \pi_\theta(\cdot \mid s)}\left[\nabla_\theta \log \pi_\theta(a \mid s) Q^{\pi_\theta}(s, a)\right]
$$

によって得られます．これはアドバンテージを使っても同じで，

$$
\nabla_\theta V^{\pi_\theta}\left(s_0\right)=\frac{1}{1-\gamma} \mathbb{E}_{s \sim d_{\sigma_0}^{\pi_\theta}} \mathbb{E}_{a \sim \pi_\theta(\cdot \mid s)}\left[\nabla_\theta \log \pi_\theta(a \mid s) Q^{\pi_\theta}(s, a)\right]
$$

も成立します．このとき，方策勾配法は収束するでしょうか？
[RL_value_function.ipynb](RL_value_function.ipynb)であるように，実は価値関数は方策に対して凸ではありません．よって，方策勾配法が収束するのは自明ではありません．


## Direct Parameterizationにおける方策勾配法の収束

$\theta_{s, a}=\pi_\theta(a \mid s)$の場合を考えます．
このとき
$$
\frac{\partial V^\pi(\mu)}{\partial \pi(a \mid s)}=\frac{1}{1-\gamma} d_\mu^\pi(s) Q^\pi(s, a),
$$
が成立します．
このとき，次のprojected gradient ascentについて考えてみましょう．

$$
\pi^{(t+1)}=P_{\Delta(\mathcal{A})^{|S|}}\left(\pi^{(t)}+\eta \nabla_\pi V^{(t)}(\mu)\right),
$$

学習率が$\eta=\frac{(1-\gamma)^3}{2 \gamma|\mathcal{A}|}$であれば，任意の分布$\rho$について，

$$
\min _{t<T}\left\{V^{\star}(\rho)-V^{(t)}(\rho)\right\} \leq \epsilon \quad \text { whenever } \quad T>\frac{64 \gamma|\mathcal{S} \| \mathcal{A}|}{(1-\gamma)^6 \epsilon^2}\left\|\frac{d d_\rho^{\pi^{\star}}}{\mu}\right\|_{\infty}^2 \text {. }
$$

が成立します．
ここで全イテレーションの中で一番いいやつを選んでいることに注意しましょう．Last-iterate convergenceではないですが，これはさほど問題になりません．なぜなら，方策勾配法では多くの場合でオンポリシー評価のフェーズを含むので，過去の一番良かったやつを記憶しておくことは容易だからです．

この収束の証明を見ていきましょう．

### １．Gradient dominationの証明

以下ではまず$V^{\pi_\theta}(\mu)$がPolyak-like gradient domination conditionを満たしていることを確認します．
直感的には，任意の$\theta \in \Theta$で次を満たすときに$f(\theta)$はgradient dominationを満たすといいます：

$$
f\left(\theta^{\star}\right)-f(\theta)=O(G(\theta)),
$$

ここで$\theta^\star \in \arg\max_{\theta' \in \Theta} f(\theta')$であり，$G(\theta)$は適切な一次オーダーで$\theta$が停留であることを表す項です（勾配がどれだけ大きいかを表します）．

この表記を使うとnon-convexな関数の収束を解析することができます（停留点だが最適ではない点があると困るわけですね）．
価値関数は凸ではないですが，このGradient dominationを満たします．あとはSmoothnessなどを利用して通常の勾配降下の理論を利用して方策が停留点に達することを示します．

---

任意の状態空間上の分布$\mu, \rho \in \Delta(\mathcal{S})$について，次が成立します

$$
\begin{aligned}
V^{\star}(\rho)-V^\pi(\rho) & \leq\left\|\frac{d_\rho^{\pi^*}}{d_\mu^\pi}\right\|_{\infty} \max _{\bar{\pi}}(\bar{\pi}-\pi)^{\top} \nabla_\pi V^\pi(\mu) \\
& \leq \frac{1}{1-\gamma}\left\|\frac{d_\rho^{\pi^{\star}}}{\mu}\right\|_{\infty} \max _{\bar{\pi}}(\bar{\pi}-\pi)^{\top} \nabla_\pi V^\pi(\mu)
\end{aligned}
$$

**証明**

Performance difference lemmaを使えば一瞬で出ます．
まず，

$$
\begin{aligned}
V^{\star}(\rho)-V^\pi(\rho) & =\frac{1}{1-\gamma} \sum_{s, a} d_\rho^{\pi^*}(s) \pi^{\star}(a \mid s) A^\pi(s, a) \\
& \leq \frac{1}{1-\gamma} \sum_{s, a} d_\rho^{\pi^{\star}}(s) \max _a A^\pi(s, \bar{a}) \\
& =\frac{1}{1-\gamma} \sum_s \frac{d_\rho^{\pi^*}(s)}{d_\mu^\pi(s)} \cdot d_\mu^\pi(s) \max _a A^\pi(s, \bar{a}) \\
& \leq \frac{1}{1-\gamma}\left(\max _s \frac{d_\rho^{\pi^*}(s)}{d_\mu^\pi(s)}\right) \sum_s d_\mu^\pi(s) \max _{\bar{a}} A^\pi(s, \bar{a}),
\end{aligned}
$$

です．
ここで更に

$$
\begin{aligned}
\sum_s \frac{d_\mu^\pi(s)}{1-\gamma} \max _a A^\pi(s, \bar{a}) & =\max _{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} \sum_{s, a} \frac{d_\mu^\pi(s)}{1-\gamma} \bar{\pi}(a \mid s) A^\pi(s, a) \\
& =\max _{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} \sum_{s, a} \frac{d_\mu^\pi(s)}{1-\gamma}(\bar{\pi}(a \mid s)-\pi(a \mid s)) A^\pi(s, a) \\
& =\max _{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}} \sum_{s, a} \frac{d_\mu^\pi(s)}{1-\gamma}(\bar{\pi}(a \mid s)-\pi(a \mid s)) Q^\pi(s, a) \\
& =\max _{\bar{\pi} \in \Delta(\mathcal{A})^{|\mathcal{S}|}}(\bar{\pi}-\pi)^{\top} \nabla_\pi V^\pi(\mu)
\end{aligned}
$$

であることを使えば終わりです．


### 2. Projected gradientが十分小さくなるときに$\varepsilon$-stationaryが成立することを示す．

収束レートを証明しましょう．まず次の定義を導入します．

---

**定義：First order stationary**

ある方策$\pi_\theta \in \Delta(\mathcal{A})^{|\mathcal{S}|}$は次を満たすときに$\epsilon$-stationaryと呼ぶ：

$$G(\pi_\theta) := \max_{\pi_\theta + \delta \in \Delta(\mathcal{A})^{|\mathcal{S}|}, \|\delta \|_2 \leq 1} \delta^\top \nabla_\pi V^{\pi_\theta}(\mu) \leq \epsilon$$

もし$\epsilon=0$なら，$\delta^\top \nabla_\pi V^\pi(\mu)\leq 0$です．
つまり，$\|\delta \|_2 \leq 1$を満たすような動ける範囲のうち，どの方向に動いても一次の値が変化しないわけですね．
また，$\Delta(\mathcal{A})^{|\mathcal{S}|}$が凸集合なので，$\delta$は実行可能な方向であることにも注意しましょう．

---

このとき次が成立します．

---

$V^\pi(\mu)$が$\pi$について$\beta$-smoothであるとします（後で示します）．
勾配のprojectionを

$$
G^\eta(\pi)=\frac{1}{\eta}(P_{\Delta(\mathcal{A})^{|\mathcal{S}|}}(\pi + \eta \nabla_\pi V^\pi(\mu) - \pi))
$$

とします（$P$はprojectionです）．このときprojected gradientによる更新を

$$
\pi^+ = \pi + \eta G^\eta(\pi)
$$

とします．もし$\|G^\eta(\pi)\|_2 \leq \epsilon$ならば

$$
\max_{\pi_\theta + \delta \in \Delta(\mathcal{A})^{|\mathcal{S}|}, \|\delta \|_2 \leq 1}
\delta^\top \nabla_\pi V^\pi(\mu) \leq \epsilon (\eta \beta + 1)
$$

が成立します．

**証明：[CVX_weakly_convex_and_Moreau_envelope.ipynb](CVX_weakly_convex_and_Moreau_envelope.ipynb)により一般なもの（weakly convexの場合）の証明を書きました．多分そっちのほうがわかりやすい．**

[RL_useful_lemma/Optimization_standard.ipynb](RL_useful_lemma/Optimization_standard.ipynb)の定理を使うと，
$$
\nabla_\pi V^{\pi^+}(\mu) \in N_{\Delta(\mathcal{A})^{|\mathcal{S}|}}(\pi^+) + \epsilon(\eta \beta + 1) B_2
$$
が成立します．ここで$B_2$は$\ell_2$の単位球，$N_{\Delta(\mathcal{A})^{|\mathcal{S}|}}$はnormal coneです．
よって，$\nabla_\pi V^{\pi^+}(\mu)$が$\epsilon (\eta \beta + 1)$だけnormal coneから距離があり，$\delta$はtangent coneに属します（TODO: なぜ？）．これで証明終わりです．

### 3. $\beta$-smoothnessを示す

$$G(\pi_\theta) := \max_{\pi_\theta + \delta \in \Delta(\mathcal{A})^{|\mathcal{S}|}, \|\delta \|_2 \leq 1} \delta^\top \nabla_\pi V^{(t)}(\mu) \leq \epsilon$$

とします．$V^\pi(\mu)$が$\beta$-smoothであることを示しましょう．次が成立します：
$$
\left\|\nabla_\pi V^\pi\left(s_0\right)-\nabla_\pi V^{\pi^{\prime}}\left(s_0\right)\right\|_2 \leq \frac{2 \gamma|\mathcal{A}|}{(1-\gamma)^3}\left\|\pi-\pi^{\prime}\right\|_2
$$
これの証明はそんなに難しくないので省略します．
つまり，$V^\pi(\mu)$は$\beta=\frac{2\gamma |\mathcal{A}|}{(1-\gamma)^3}$について$\beta$-smoothです．


### 4. どこかでProjected Gradientが十分小さくなる点があることを示す．
よって，$G^\eta(\pi)$と$\eta=\frac{1}{\beta}$について，[RL_useful_lemma/Optimization_standard.ipynb](RL_useful_lemma/Optimization_standard.ipynb)の定理を使うと，
$$
\min _{t=0,1, \ldots, T-1}\left\|G^\eta\left(\pi^{(t)}\right)\right\|_2 \leq \frac{\sqrt{2 \beta\left(V^{\star}(\mu)-V^{(0)}(\mu)\right)}}{\sqrt{T}}
$$

が成立します．

### 5. 合体する

よって，上で示したStationary pointの定理から，
$$
\min _{t=0,1, \ldots, T} \max _{\pi^{(t)}+\delta \in \Delta(A)^{|\mathcal{S}|},\|\delta\|_2 \leq 1} \delta^{\top} \nabla_\pi V^{\pi^{(t+1)}}(\mu) \leq(\eta \beta+1) \frac{\sqrt{2 \beta\left(V^{\star}(\mu)-V^{(0)}(\mu)\right)}}{\sqrt{T}}
$$
が成立します．
ここで
$$
\begin{aligned}
\max _{\bar{\pi} \in \Delta(A)^{|\mathcal{S}|}}(\bar{\pi}-\pi)^{\top} \nabla_\pi V^\pi(\mu) & =2 \sqrt{|\mathcal{S}|} \max _{\bar{\pi} \in \Delta(A)^{|\mathcal{S}|}} \frac{1}{2 \sqrt{|\mathcal{S}|}}(\bar{\pi}-\pi)^{\top} \nabla_\pi V^\pi(\mu) \\
& \leq 2 \sqrt{|\mathcal{S}|} \max _{\pi+\delta \in \Delta(A)^{\mathcal{S} \mid},\|\delta\|_2 \leq 1} \delta^{\top} \nabla_\pi V^\pi(\mu)
\end{aligned}
$$
なので，Gradient dominanceのやつから
$$
\min _{t=0,1, \ldots, T} V^{\star}(\rho)-V^{(t)}(\rho) \leq \frac{4 \sqrt{|\mathcal{S}|}}{1-\gamma}\left\|\frac{d_\rho^{\pi^{\star}}}{\mu}\right\|_{\infty} \frac{\sqrt{2 \beta\left(V^{\star}(\mu)-V^{(0)}(\mu)\right)}}{\sqrt{T}}
$$
であり，あとは変形すれば終わりです．

# 自然勾配法の収束レートについて

参考：
* [Reinforcement Learning: Theory and Algorithms](https://rltheorybook.github.io/)の12.4章あたり
* [Fast Global Convergence of Natural Policy Gradient Methods with Entropy Regularization](https://arxiv.org/abs/2007.06558)


## エントロピー正則化と自然勾配法

エントロピー正則化を導入すると自然方策勾配は線形収束します．見ていきましょう．

参考：
* [Fast Global Convergence of Natural Policy Gradient Methods with Entropy Regularization](https://arxiv.org/abs/2007.06558)

表記：
* 割引訪問分布：
  * $\forall s \in \mathcal{S}: \quad d_{s_0}^\pi(s):=(1-\gamma) \sum_{t=0}^{\infty} \gamma^t \mathbb{P}\left(s_t=s \mid s_0\right)$,
  * $\forall s \in \mathcal{S}: \quad d_\rho^\pi(s):=\mathbb{E}_{s_0 \sim \rho}\left[d_{s_0}^\pi(s)\right]$
* ソフトマックス方策：
$$
\pi_\theta:=\operatorname{softmax}(\theta) \quad \text { or } \quad \pi_\theta(a \mid s):=\frac{\exp (\theta(s, a))}{\sum_{a^{\prime} \in \mathcal{A}} \exp \left(\theta\left(s, a^{\prime}\right)\right)}
$$
* エントロピー正則化：$\mathcal{H}(\rho, \pi):=\underset{\substack{s_0 \sim \rho, a_t \sim \pi\left(\cdot \mid s_t\right), s_{t+1} \sim P\left(\cdot \mid s_t, a_t\right), \forall t \geq 0}}{\mathbb{E}}\left[\sum_{t=0}^{\infty}-\gamma^t \log \pi\left(a_t \mid s_t\right)\right]=\frac{1}{1-\gamma} \underset{s \sim d_\rho^\pi}{\mathbb{E}}\left[\sum_{a \in \mathcal{A}} \pi(a \mid s) \log \frac{1}{\pi(a \mid s)}\right]$
    * 報酬：$r_\tau(s, a):=r(s, a)-\tau \log \pi(a \mid s)$
    * Q関数：$Q_\tau^\pi(s, a)=r(s, a)+\gamma \mathbb{E}_{s^{\prime} \sim P(\cdot \mid s, a)}\left[V_\tau^\pi\left(s^{\prime}\right)\right]$
    * V関数：$V_\tau^\pi(s)=\mathbb{E}_{a \sim \pi(\cdot \mid s)}\left[-\tau \log \pi(a \mid s)+Q_\tau^\pi(s, a)\right]$
    * アドバンテージ関数：$A_\tau^\pi(s, a):=Q_\tau^\pi(s, a)-\tau \log \pi(a \mid s)-V_\tau^\pi(s)$


### ソフトマックス方策での自然勾配法の更新式

ソフトマックス方策のもとで自然勾配法を考えてみましょう．つまり，
$$
\theta \leftarrow \theta+\eta\left(\mathcal{F}_\rho^\theta\right)^{\dagger} \nabla_\theta V_\tau^{\pi_\theta}(\rho)
$$
です．このとき，自然勾配法は
$$
\pi^{(t+1)}(a \mid s)=\frac{1}{Z^{(t)}(s)}\left(\pi^{(t)}(a \mid s)\right)^{1-\frac{\eta \tau}{1-\gamma}} \exp \left(\frac{\eta Q_\tau^{\pi^{(t)}}(s, a)}{1-\gamma}\right)
$$
を満たします．証明は論文のAppendix A.1に任せます（TODO: 今度書く）

また，学習率が$\eta=\frac{1-\gamma}{\tau}$を満たすとき，
$$
\pi^{(t+1)}(\cdot \mid s)=\frac{1}{Z^{(t)}(s)} \exp \left(\frac{Q_\tau^{\pi^{(t)}}(s, \cdot)}{\tau}\right) \quad \text { when } \eta=\frac{1-\gamma}{\tau}
$$
になることも抑えておきましょう．$\tau\approx 0$のとき，これはほぼ貪欲方策になります．


### エントロピー正則化された自然方策勾配の収束レート

任意の学習率
$
0<\eta \leq(1-\gamma) / \tau
$
について，次が成立します：

$$
\begin{aligned}
\left\|Q_\tau^{\star}-Q_\tau^{(t+1)}\right\|_{\infty} & \leq C_1 \gamma(1-\eta \tau)^t \\
\left\|\log \pi_\tau^{\star}-\log \pi^{(t+1)}\right\|_{\infty} & \leq 2 C_1 \tau^{-1}(1-\eta \tau)^t
\end{aligned}
$$

ここで
$
C_1:=\left\|Q_\tau^{\star}-Q_\tau^{(0)}\right\|_{\infty}+2 \tau\left(1-\frac{\eta \tau}{1-\gamma}\right)\left\|\log \pi_\tau^{\star}-\log \pi^{(0)}\right\|_{\infty}
$
です．

ここからいくつか言えることがあります：
1. $\left\|Q_\tau^{\star}-Q_\tau^{(t)}\right\|_{\infty} \leq \epsilon$は$\frac{1}{\eta \tau} \log \left(\frac{C_1 \gamma}{\epsilon}\right)$回以上のイテレーションで収束します．
    * この収束は$S$や$A$のサイズに依らないことに注意しましょう．NPGもdimension freeです．
    * また，エントロピー正則化項$\tau$が大きいと収束が早くなります．つまりエントロピーが大きいほうが学習が簡単なんですね．
    * エントロピー正則化の影響を小さくする場合，つまり$\tau = \frac{\varepsilon}{1-\gamma}$の場合は$\frac{1}{(1-\gamma)\eta \varepsilon} \log \left(\frac{C_1 \gamma}{\epsilon}\right)$回程度のイテレーションになります．
2. エントロピー正則化が入る場合は最適方策が唯一存在するので，$\left\|\log \pi_\tau^{\star}-\log \pi^{(t+1)}\right\|_{\infty} \leq \epsilon$もバウンドすることができます．
3. V関数も収束します．実際，Performance difference lemmaを使えば次が成立します：
$$\begin{aligned}\left|V_\tau^{\star}(s)-V_\tau^{(t+1)}(s)\right| & =\underset{a \sim \pi^{(t+1)}(\cdot \mid s)}{\mathbb{E}}\left[\left(-\tau \log \pi_\tau^{\star}(a \mid s)+Q_\tau^{\star}(s, a)\right)-\left(-\tau \log \pi_\tau^{(t+1)}(a \mid s)+Q_\tau^{(t+1)}(s, a)\right)\right] \\ & \leq \tau\left\|\log \pi_\tau^{\star}-\log \pi_\tau^{(t+1)}\right\|_{\infty}+\left\|Q_\tau^{\star}-Q_\tau^{(t+1)}\right\|_{\infty}\end{aligned}$$
4. $\eta=(1-\gamma) / \tau$のとき，$\left\|Q_\tau^{\star}-Q_\tau^{(t)}\right\|_{\infty} \leq \epsilon$ を獲得するイテレーションの回数は
$$
\frac{1}{1-\gamma} \log \left(\frac{\gamma\left\|Q_\tau^{\star}-Q_\tau^{(0)}\right\|_{\infty}}{\epsilon}\right)
$$
以上であれば大丈夫です．また，この更新方法はSoft Policy Iteration (SPI)と同じです．


### 収束レートの証明

まず次のPerformance improvement が成立します．

---

任意の学習率$0<\eta \leq(1-\gamma) / \tau$について，

$$
V_\tau^{(t+1)}(\rho)-V_\tau^{(t)}(\rho)=\underset{s \sim d_\rho^{(t+1)}}{\mathbb{E}}\left[\left(\frac{1}{\eta}-\frac{\tau}{1-\gamma}\right) \mathrm{KL}\left(\pi^{(t+1)}(\cdot \mid s) \| \pi^{(t)}(\cdot \mid s)\right)+\frac{1}{\eta} \operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \pi^{(t+1)}(\cdot \mid s)\right)\right] .
$$

右辺がKLダイバージェンスなので，次も成立します：
$$
Q_\tau^{(t+1)}(s, a)=r(s, a)+\gamma \underset{s^{\prime} \sim P(\cdot \mid s, a)}{\mathbb{E}}\left[V_\tau^{(t+1)}\left(s^{\prime}\right)\right] \geq r(s, a)+\gamma \underset{s^{\prime} \sim P(\cdot \mid s, a)}{\mathbb{E}}\left[V_\tau^{(t)}\left(s^{\prime}\right)\right]=Q_\tau^{(t)}(s, a)
$$

**証明**（以下でも示せますが，多分Value difference lemmaを先に示したほうが簡潔ですね．）

更新方法から，

$$
\log \pi^{(t+1)}(a \mid s)=\left(1-\frac{\eta \tau}{1-\gamma}\right) \log \pi^{(t)}(a \mid s)+\frac{\eta}{1-\gamma} Q_\tau^{(t)}(s, a)-\log Z^{(t)}(s)
$$

が成り立ちます．これを書き換えると

$$
-\tau \log \pi^{(t)}(a \mid s)+Q_\tau^{(t)}(s, a)=\frac{1-\gamma}{\eta}\left(\log \pi^{(t+1)}(a \mid s)-\log \pi^{(t)}(a \mid s)\right)+\frac{1-\gamma}{\eta} \log Z^{(t)}(s)
$$

です．よって，

$$
\begin{aligned}
V_\tau^{(t)}\left(s_0\right) & =\underset{a_0 \sim \pi^{(t)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[-\tau \log \pi^{(t)}\left(a_0 \mid s_0\right)+Q_\tau^{(t)}\left(s_0, a_0\right)\right] \\
& =\underset{a_0 \sim \pi^{(t)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[\frac{1-\gamma}{\eta} \log Z^{(t)}\left(s_0\right)\right]+\underset{a_0 \sim \pi^{(t)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[\frac{1-\gamma}{\eta}\left(\log \pi^{(t+1)}\left(a_0 \mid s_0\right)-\log \pi^{(t)}\left(a_0 \mid s_0\right)\right)\right] \\
& =\frac{1-\gamma}{\eta} \log Z^{(t)}\left(s_0\right)-\frac{1-\gamma}{\eta} \operatorname{KL}\left(\pi^{(t)}\left(\cdot \mid s_0\right) \| \pi^{(t+1)}\left(\cdot \mid s_0\right)\right) \\
& =\underset{a_0 \sim \pi^{(t+1)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[\frac{1-\gamma}{\eta} \log Z^{(t)}\left(s_0\right)\right]-\frac{1-\gamma}{\eta} \operatorname{KL}\left(\pi^{(t)}\left(\cdot \mid s_0\right) \| \pi^{(t+1)}\left(\cdot \mid s_0\right)\right)
\end{aligned}
$$

です．$\log Z$を改めて書き換えると，

$$
\begin{aligned}
& V_\tau^{(t)}\left(s_0\right)= \underset{a_0 \sim \pi^{(t+1)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[-\tau \log \pi^{(t+1)}\left(a_0 \mid s_0\right)+Q_\tau^{(t)}\left(s_0, a_0\right)+\left(\tau-\frac{1-\gamma}{\eta}\right)\left(\log \pi^{(t+1)}\left(a_0 \mid s_0\right)-\log \pi^{(t)}\left(a_0 \mid s_0\right)\right)\right] \\
& \quad-\frac{1-\gamma}{\eta} \operatorname{KL}\left(\pi^{(t)}\left(\cdot \mid s_0\right) \| \pi^{(t+1)}\left(\cdot \mid s_0\right)\right) \\
&=\underset{a_0 \sim \pi^{(t+1)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[-\tau \log \pi^{(t+1)}\left(a_0 \mid s_0\right)+Q_\tau^{(t)}\left(s_0, a_0\right)\right]+\left(\tau-\frac{1-\gamma}{\eta}\right) \operatorname{KL}\left(\pi^{(t+1)}\left(\cdot \mid s_0\right) \| \pi^{(t)}\left(\cdot \mid s_0\right)\right) \\
& \quad-\frac{1-\gamma}{\eta} \operatorname{KL}\left(\pi^{(t)}\left(\cdot \mid s_0\right) \| \pi^{(t+1)}\left(\cdot \mid s_0\right)\right) \\
&=\underset{\substack{\left.a_0 \sim \pi^{(t+1)}\left(\cdot \mid s_0\right)\right) \\
s_1 \sim P\left(\cdot \mid s_0, a_0\right)}}{\mathbb{E}}\left[-\tau \log \pi^{(t+1)}\left(a_0 \mid s_0\right)+r\left(s_0, a_0\right)+\gamma V_\tau^{(t)}\left(s_1\right)\right] \\
& \quad-\left(\frac{1-\gamma}{\eta}-\tau\right) \operatorname{KL}\left(\pi^{(t+1)}\left(\cdot \mid s_0\right) \| \pi^{(t)}\left(\cdot \mid s_0\right)\right)-\frac{1-\gamma}{\eta} \operatorname{KL}\left(\pi^{(t)}\left(\cdot \mid s_0\right) \| \pi^{(t+1)}\left(\cdot \mid s_0\right)\right)
\end{aligned}
$$

あとは$V^{(t)}_\tau$の項を再帰的に展開していけば大丈夫です．

---



続いて，更新がSoft Bellman 最適作用素に対応することを示しましょう．Soft Bellman作用素は次のように定義されます：

$$
\mathcal{T}_\tau(Q)(s, a):=r(s, a)+\gamma \underset{s^{\prime} \sim P(\cdot \mid s, a)}{\mathbb{E}}\left[\max _{\pi\left(\cdot \mid s^{\prime}\right) \in \Delta(\mathcal{A})} \underset{a^{\prime} \sim \pi\left(\cdot \mid s^{\prime}\right)}{\mathbb{E}}\left[Q\left(s^{\prime}, a^{\prime}\right)-\tau \log \pi\left(a^{\prime} \mid s^{\prime}\right)\right]\right]
$$

また，これは
$$
\mathcal{T}_\tau(Q)(s, a)=r(s, a)+\gamma \underset{s^{\prime} \sim P(\cdot \mid s, a)}{\mathbb{E}}\left[\tau \log \left(\left\|\exp \left(Q\left(s^{\prime}, \cdot\right) / \tau\right)\right\|_1\right)\right]
$$
を満たし，Contraction $\left\|\mathcal{T}_\tau\left(Q_1\right)-\mathcal{T}_\tau\left(Q_2\right)\right\|_{\infty} \leq \gamma\left\|Q_1-Q_2\right\|_{\infty}$が成立します．


---

**$\eta = (1-\gamma) / \tau$のとき**

$$
\begin{aligned}
&Q_\tau^{(t+1)}(s, a) \stackrel{(\mathrm{i})}{=} r(s, a)+\gamma \underset{s^{\prime} \sim P(\cdot \mid s, a)}{\mathbb{E}}\left[V_\tau^{(t+1)}\left(s^{\prime}\right)\right]\\
& \stackrel{\text { (ii) }}{=} r(s, a)+\gamma \underset{\substack{s^{\prime} \sim P(\cdot \mid s, a), a^{\prime} \sim \pi^{(t+1)}\left(\cdot \mid s^{\prime}\right)}}{\mathbb{E}}\left[-\tau \log \pi^{(t+1)}\left(a^{\prime} \mid s^{\prime}\right)+Q_\tau^{(t+1)}\left(s^{\prime}, a^{\prime}\right)\right] \\
& \stackrel{\text { (iii) }}{\geq} r(s, a)+\gamma \underset{\substack{s^{\prime} \sim P(\cdot \mid s, a), a^{\prime} \sim \pi^{(t+1)}\left(\cdot \mid s^{\prime}\right)}}{\mathbb{E}}\left[-\tau \log \pi^{(t+1)}\left(a^{\prime} \mid s^{\prime}\right)+Q_\tau^{(t)}\left(s^{\prime}, a^{\prime}\right)\right] \\
& \stackrel{\text { (iv) }}{=} r(s, a)+\gamma \underset{s^{\prime} \sim P(\cdot \mid s, a)}{\mathbb{E}}\left[\tau \log \left(\left\|\exp \left(Q^{(t)}\left(s^{\prime}, \cdot\right) / \tau\right)\right\|_1\right)\right] \\
& \stackrel{\text { (v) }}{=} \mathcal{T}_\tau\left(Q_\tau^{(t)}\right)(s, a) .
\end{aligned}
$$

(iii)は上で示したMonotonic performance improvementから成立します．
以上から
$$
0 \leq Q_\tau^{\star}-Q_\tau^{(t+1)} \leq Q_\tau^{\star}-\mathcal{T}_\tau\left(Q_\tau^{(t+1)}\right)
$$
が成立するので，
$$
\begin{aligned}
\left\|Q_\tau^{\star}-Q_\tau^{(t+1)}\right\|_{\infty} & \leq\left\|Q_\tau^{\star}-\mathcal{T}_\tau\left(Q_\tau^{(t)}\right)\right\|_{\infty}=\left\|\mathcal{T}_\tau\left(Q_\tau^{\star}\right)-\mathcal{T}_\tau\left(Q_\tau^{(t)}\right)\right\|_{\infty} \leq \gamma\left\|Q_\tau^{\star}-Q_\tau^{(t)}\right\|_{\infty} \\
& \leq \gamma^{t+1}\left\|Q_\tau^{\star}-Q_\tau^{(0)}\right\|_{\infty},
\end{aligned}
$$
がすぐにわかります．

---

**一般の学習率のとき**

TODO: 証明書く


## CPIスタイルの更新について

次の更新を考えましょう：

$$
\pi^{(t+1)}=(1-\beta) \pi^{(t)}+\beta \bar{\pi}^{(t+1)}
$$

ここで$\bar{\pi}^{(t+1)}$はSPIで更新された方策であり，
$$
\bar{\pi}^{(t+1)}(a \mid s)=\frac{1}{\bar{Z}^{(t)}(s)} \exp \left(Q_\tau^{(t)}(s, a) / \tau\right)
$$
とします．また，$\bar{Z}^{(t)}(s)=\sum_{a \in \mathcal{A}} \exp \left(Q_\tau^{(t)}(s, a) / \tau\right) \quad$ and $\quad Q_\tau^{(t)}=Q_\tau^{\pi^{(t)}}$です．

このとき，次の収束レートが成立します：

$$
V_\tau^{\star}(\rho)-V_\tau^{(t)}(\rho) \leq\left\|\frac{\rho}{\mu_\tau^{\star}}\right\|_{\infty}(1-\beta(1-\gamma))^t\left(V_\tau^{\star}\left(\mu_\tau^{\star}\right)-V_\tau^{(0)}\left(\mu_\tau^{\star}\right)\right), \quad \forall t \geq 0,
$$

ここで$\mu_\tau^{\star}=d_{\mu_\tau^{\star}}^{\pi_\tau^{\star}}$は$\pi_\tau^\star$による定常分布です．
証明しましょう．

**証明**

まず次のPerformance improvementが成立します：

---

任意の$\rho$について，
$$
V_\tau^{(t+1)}(\rho)-V_\tau^{(t)}(\rho) \geq \frac{\beta \tau}{1-\gamma} \underset{s \sim d_\rho^{(t+1)}}{\mathbb{E}}\left[\operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)\right]
$$
が成立します．

**証明**

まず，$\bar{\pi}^{(t+1)}$の定義から
$$
\begin{aligned}
V_\tau^{(t)}\left(s_0\right) & =\underset{a_0 \sim \pi^{(t)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[-\tau \log \pi^{(t)}\left(a_0 \mid s_0\right)+Q_\tau^{(t)}\left(s_0, a_0\right)\right] \\
& =\underset{a_0 \sim \pi^{(t)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[-\tau \log \pi^{(t)}\left(a_0 \mid s_0\right)+\tau \log \bar{\pi}^{(t+1)}\left(a_0 \mid s_0\right)\right]+\tau \log \bar{Z}^{(t)}\left(s_0\right) \\
& =\tau \log \bar{Z}^{(t)}\left(s_0\right)-\tau \operatorname{KL}\left(\pi^{(t)}\left(\cdot \mid s_0\right) \| \bar{\pi}^{(t+1)}\left(\cdot \mid s_0\right)\right) \\
& =\tau \underset{a_0 \sim \pi^{(t+1)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[\log \bar{Z}^{(t)}\left(s_0\right)\right]-\tau \operatorname{KL}\left(\pi^{(t)}\left(\cdot \mid s_0\right) \| \bar{\pi}^{(t+1)}\left(\cdot \mid s_0\right)\right),
\end{aligned}
$$

が成立します．あとは$\pm$項を考えれば，
$$
\begin{aligned}
V_\tau^{(t)}\left(s_0\right)= & \underset{a_0 \sim \pi^{(t+1)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[\tau \log \bar{Z}^{(t)}\left(s_0\right)-\tau \log \pi^{(t+1)}\left(a_0 \mid s_0\right)+\tau \log \bar{\pi}^{(t+1)}\left(a_0 \mid s_0\right)\right] \\
& +\tau \mathrm{KL}\left(\pi^{(t+1)}\left(\cdot \mid s_0\right) \| \bar{\pi}^{(t+1)}\left(\cdot \mid s_0\right)\right)-\tau \mathrm{KL}\left(\pi^{(t)}\left(\cdot \mid s_0\right) \| \bar{\pi}^{(t+1)}\left(\cdot \mid s_0\right)\right) \\
= & \underset{a_0 \sim \pi^{(t+1)}\left(\cdot \mid s_0\right)}{\mathbb{E}}\left[-\tau \log \pi^{(t+1)}\left(a_0 \mid s_0\right)+Q_\tau^{(t)}\left(s_0, a_0\right)\right]+\tau \mathrm{KL}\left(\pi^{(t+1)}\left(\cdot \mid s_0\right) \| \bar{\pi}^{(t+1)}\left(\cdot \mid s_0\right)\right) \\
& \quad-\tau \mathrm{KL}\left(\pi^{(t)}\left(\cdot \mid s_0\right) \| \bar{\pi}^{(t+1)}\left(\cdot \mid s_0\right)\right) \\
= & V_\tau^{(t+1)}\left(s_0\right)+\frac{\tau}{1-\gamma} \underset{s \sim d_{s_0}^{(t+1)}}{\mathbb{E}}\left[\operatorname{KL}\left(\pi^{(t+1)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)-\operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)\right] .
\end{aligned}
$$

であり，
$$
V_\tau^{(t+1)}(\rho)-V_\tau^{(t)}(\rho)=\frac{\tau}{1-\gamma} \underset{s \sim d_\rho^{(t+1)}}{\mathbb{E}}\left[\operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)-\operatorname{KL}\left(\pi^{(t+1)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)\right]
$$
であることがすぐにわかります．

ここで，KLの凸性を使い，Jensenの不等式を使えば，
$$
\begin{aligned}
\operatorname{KL}\left(\pi^{(t+1)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right) & \leq \beta \operatorname{KL}\left(\bar{\pi}^{(t+1)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)+(1-\beta) \operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right) \\
& =(1-\beta) \operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right) .
\end{aligned}
$$
であることはすぐにわかります．
上の式を合体して，

$$
V_\tau^{(t+1)}(\rho)-V_\tau^{(t)}(\rho) \geq \frac{\beta \tau}{1-\gamma} \underset{s \sim d_\rho^{(t+1)}}{\mathbb{E}}\left[\operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)\right]
$$

が成立します．

**補足**

これはoccupancy measureの定義を使えば
$$
\begin{aligned}
V_\tau^{(t+1)}(\rho)-V_\tau^{(t)}(\rho) 
&\geq \frac{\beta \tau}{1-\gamma} \underset{s \sim d_\rho^{(t+1)}}{\mathbb{E}}\left[\operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)\right]\\
&\geq \beta \tau \underset{s \sim \rho}{\mathbb{E}}\left[\operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)\right]\\
\end{aligned}
$$

として良いはず．[RL theory and algorithms](https://rltheorybook.github.io/rltheorybook_AJKS.pdf#page=131.19)の139ページあたり参照．
これに初期状態分布として$d_\rho^{(t+1)}$を入れると，
$$
\begin{aligned}
V_\tau^{(t+1)}(\rho)-V_\tau^{(t)}(\rho) 
&\geq \beta \tau \underset{s \sim d_\rho^{\pi^\star}}{\mathbb{E}}\left[\operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)\right]\\
\end{aligned}
$$
が成立する．これからやる処理もこれを使ってるよ（多分$\sum^T$みたいな処理で打ち消しあう証明方法ならこいつは影響しない）．

---

Performance improvementを使うと，

$$
\begin{aligned}
V_\tau^{\star}(\rho)-V_\tau^{(t+1)}(\rho) & =V_\tau^{\star}(\rho)-V_\tau^{(t)}(\rho)+\left(V_\tau^{(t)}(\rho)-V_\tau^{(t+1)}(\rho)\right) \\
& \stackrel{(\mathrm{i})}{\leq} V_\tau^{\star}(\rho)-V_\tau^{(t)}(\rho)-\frac{\beta \tau}{1-\gamma} \underset{s \sim d_\rho^{(t+1)}}{\mathbb{E}}\left[\operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)\right] \\
& \stackrel{(\mathrm{ii})}{\leq} V_\tau^{\star}(\rho)-V_\tau^{(t)}(\rho)-\frac{\beta \tau}{1-\gamma}\left\|\frac{d_\rho^{\pi_\tau^{\star}}}{d_\rho^{(t+1)}}\right\|_{\infty}^{-1} \underset{s \sim d_\rho^{\pi *}}{\mathbb{E}}\left[\operatorname{KL}\left(\pi^{(t)}(\cdot \mid s) \| \bar{\pi}^{(t+1)}(\cdot \mid s)\right)\right] \\
& \stackrel{(\mathrm{iii})}{\leq} V_\tau^{\star}(\rho)-V_\tau^{(t)}(\rho)-\beta\left\|\frac{d_\rho^{\pi_\tau^{\star}}}{d_\rho^{(t+1)}}\right\|_{\infty}^{-1}\left(V_\tau^{\star}(\rho)-V_\tau^{(t)}(\rho)\right) \\
& =\left(1-\beta\left\|\frac{d_\rho^{\pi_\tau^{\star}}}{d_\rho^{(t+1)}}\right\|_{\infty}^{-1}\right)\left(V_\tau^{\star}(\rho)-V_\tau^{(t)}(\rho)\right) .
\end{aligned}
$$

が成立することがすぐにわかります．ここで(iii)は上の方でやった（CPIを使わない）Performance improvementの式を使いました．
最後に$\rho$が$\mu_\tau^\star$である場合について考えれば，
$$
\begin{aligned}
V_\tau^{\star}\left(\mu_\tau^{\star}\right)-V_\tau^{(t+1)}\left(\mu_\tau^{\star}\right) & \leq\left(1-\beta\left\|\frac{d_{\mu_\tau^{\star}}^{\pi_\tau^{\star}}}{d_{\mu_\tau^{\star}}^{(t+1)}}\right\|_{\infty}^{-1}\right)\left(V_\tau^{\star}\left(\mu_\tau^{\star}\right)-V_\tau^{(t)}\left(\mu_\tau^{\star}\right)\right) \\
& \leq\left(1-\beta\left\|\frac{\mu_\tau^{\star}}{(1-\gamma) \mu_\tau^{\star}}\right\|_{\infty}^{-1}\right)\left(V_\tau^{\star}\left(\mu_\tau^{\star}\right)-V_\tau^{(t)}\left(\mu_\tau^{\star}\right)\right) \\
& =(1-\beta(1-\gamma))\left(V_\tau^{\star}\left(\mu_\tau^{\star}\right)-V_\tau^{(t)}\left(\mu_\tau^{\star}\right)\right)\\
& \cdots\\
& \leq(1-\beta(1-\gamma))^t\left(V_\tau^{\star}\left(\mu_\tau^{\star}\right)-V_\tau^{(0)}\left(\mu_\tau^{\star}\right)\right)\\
\end{aligned}
$$
が成立します．

よって
$$
\begin{aligned}
V_\tau^{\star}\left(\rho\right)-V_\tau^{(t+1)}\left(\rho\right) 
&\leq 
\left\|
\frac{\rho}{\mu_\tau^\star}
\right\|_\infty
\left( V_\tau^{\star}\left(\mu_\tau^\star\right)-V_\tau^{(t+1)}\left(\mu_\tau^\star\right) \right)
\\
& 
\leq
\left\|
\frac{\rho}{\mu_\tau^\star}
\right\|_\infty
(1-\beta(1-\gamma))^t\left(V_\tau^{\star}\left(\mu_\tau^{\star}\right)-V_\tau^{(0)}\left(\mu_\tau^{\star}\right)\right)\\
\end{aligned}
$$
